In [1]:
import pandas as pd
import numpy as np

In [2]:
auth_params = {"email":"francolsimonetti@gmail.com","password":"pAgaMFf6egSAW5T"}

In [ ]:
# api_host = "http://localhost:5050/api"

# api_key = None
# s = requests.Session()
# try:
#     r = s.post(api_host+'/auth/', data=auth_params)
#     if(r.status_code == 200):
#         #Lets store the api key in a new variable and use it again in new requests
#         json_response = r.json()
#         api_key = json_response.get("token")
#         print(api_key + "This is your user API key.") #Comment this line if you don't want your API key to show up in the terminal
#     else:
#         print(r.status_code)
#         print(r.text)
# except requests.exceptions.RequestException as req_ex:
#     print(req_ex)
#     print("Something went wrong with the request.")

# if api_key:
#     #Add the api key to the requests headers of the requests Session object in order to use the restricted endpoints.
#     s.headers.update({"Authorization": "Bearer %s" % api_key}) 
#     #Lets get all the diseases associated to a gene eg. APP (EntrezID 351) and restricted by a source.
#     gda_response = s.get(api_host+'/gda/gene/351', params={'source':'UNIPROT'})
#     print(gda_response.json())

# if s:
#     s.close()

In [6]:
# este dataset viene de la pagina de downloads (https://www.disgenet.org/downloads; Octubre 2020)
vda = pd.read_csv('../../data/curated_variant_disease_associations.tsv.gz', sep='\t')
vda.columns = vda.columns.str.lower().str.replace(' ',"_").str.replace("-",'_').str.replace('/','_')
vda = vda.rename(columns={'snpid':'snp_id'})

In [7]:
print(f'snps unicos en el dataset: {len(vda.snp_id.unique())} vs {vda.shape}')

snps unicos en el dataset: 168051 vs (261227, 16)


In [9]:
# Mappings UniProts: uniprots id con entrez gene id (https://www.disgenet.org/downloads, Mappings)
uniprots = pd.read_csv('../../data/mapa_geneid_4_uniprot_crossref.tsv.gz', sep='\t', compression='gzip')
uniprots = uniprots.rename(columns={'UniProtKB':'uniprot_acc', 'GENEID':'gene_id'})

In [10]:
# The file contains the mappings of DisGeNET variants (dbSNP Identifiers)
# to NCBI Entrez identifiers according to dbSNP database (https://www.disgenet.org/downloads, Variant-Gene Mappings File)
variant_gene = pd.read_csv('variant_to_gene_mappings.tsv.gz', sep='\t', compression='gzip')
variant_gene = variant_gene.rename(columns={'snpId':'snp_id', 'geneId':'gene_id', 'geneSymbol': 'gene_name', 'sourceId':'source_id'})

In [11]:
uniprots.head()

,uniprot_acc,gene_id
0,P04217,1
1,P11245,10
2,P00813,100
3,P19022,1000
4,Q9Y243,10000


In [12]:
variant_gene.head()

,snp_id,gene_id,gene_name,source_id
0,rs7980288,2,A2M,VEP
1,rs7980288,2,A2M,DBSNP
2,rs3832852,2,A2M,DBSNP
3,rs669,2,A2M,VEP
4,rs669,2,A2M,DBSNP


In [13]:
# agrego los uniprots
variant_gene = variant_gene.merge(uniprots)
variant_gene.head()

,snp_id,gene_id,gene_name,source_id,uniprot_acc
0,rs7980288,2,A2M,VEP,P01023
1,rs7980288,2,A2M,DBSNP,P01023
2,rs3832852,2,A2M,DBSNP,P01023
3,rs669,2,A2M,VEP,P01023
4,rs669,2,A2M,DBSNP,P01023


In [16]:
vda

,snp_id,chromosome,position,dsi,dpi,diseaseid,diseasename,diseasetype,diseaseclass,diseasesemantictype,score,ei,yearinitial,yearfinal,nofpmids,source
0,rs1000005,21,33060745,NaN,NaN,C0200638,Eosinophil count procedure,phenotype,NaN,Laboratory Procedure,0.7,1.0,2016.0,2016.0,1,GWASCAT
1,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,0.7,1.0,2017.0,2017.0,1,GWASCAT
2,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,0.7,1.0,2007.0,2007.0,1,CLINVAR
3,rs1000096,4,38691214,NaN,NaN,C1305855,Body mass index,phenotype,NaN,Clinical Attribute,0.7,1.0,2019.0,2019.0,1,GWASCAT
4,rs10001106,4,10125817,0.925,0.12,C0003868,"Arthritis, Gouty",disease,C16;C18;C05,Disease or Syndrome,0.7,1.0,2010.0,2013.0,3,GWASDB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261222,rs9999118,4,123130312,1.000,0.04,C0022104,Irritable Bowel Syndrome,disease,C06,Disease or Syndrome,0.8,1.0,2014.0,2014.0,1,GWASCAT;GWASDB
261223,rs999921351,8,86628939,1.000,0.12,C1849792,Achromatopsia 3,disease,C23;C11;C10,Disease or Syndrome,0.7,1.0,2017.0,2017.0,1,CLINVAR
261224,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,0.8,1.0,2009.0,2009.0,1,GWASCAT;GWASDB
261225,rs999944,2,64822719,1.000,0.08,C0520679,"Sleep Apnea, Obstructive",disease,C08;C10,Disease or Syndrome,0.7,1.0,2016.0,2016.0,1,GWASCAT


In [17]:
disgenet_total = vda.merge(variant_gene, on=['snp_id'], how='left')
disgenet_total

,snp_id,chromosome,position,dsi,dpi,diseaseid,diseasename,diseasetype,diseaseclass,diseasesemantictype,score,ei,yearinitial,yearfinal,nofpmids,source,gene_id,gene_name,source_id,uniprot_acc
0,rs1000005,21,33060745,NaN,NaN,C0200638,Eosinophil count procedure,phenotype,NaN,Laboratory Procedure,0.7,1.0,2016.0,2016.0,1,GWASCAT,NaN,NaN,NaN,NaN
1,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,0.7,1.0,2017.0,2017.0,1,GWASCAT,8821.0,INPP4B,DBSNP,O15327
2,rs10000770,4,142693109,1.000,0.04,C0023467,"Leukemia, Myelocytic, Acute",disease,C04,Neoplastic Process,0.7,1.0,2017.0,2017.0,1,GWASCAT,8821.0,INPP4B,VEP,O15327
3,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,0.7,1.0,2007.0,2007.0,1,CLINVAR,3508.0,IGHMBP2,VEP,P38935
4,rs1000091588,11,68935374,1.000,0.12,C1858517,SPINAL MUSCULAR ATROPHY WITH RESPIRATORY DISTR...,disease,C16;C08;C10,Disease or Syndrome,0.7,1.0,2007.0,2007.0,1,CLINVAR,3508.0,IGHMBP2,DBSNP,P38935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489665,rs999921351,8,86628939,1.000,0.12,C1849792,Achromatopsia 3,disease,C23;C11;C10,Disease or Syndrome,0.7,1.0,2017.0,2017.0,1,CLINVAR,54714.0,CNGB3,VEP,Q9NQW8
489666,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,0.8,1.0,2009.0,2009.0,1,GWASCAT;GWASDB,3710.0,ITPR3,VEP,Q14573
489667,rs999943,6,33656956,1.000,0.08,C0028754,Obesity,disease,C23;C18,Disease or Syndrome,0.8,1.0,2009.0,2009.0,1,GWASCAT;GWASDB,3710.0,ITPR3,DBSNP,Q14573
489668,rs999944,2,64822719,1.000,0.08,C0520679,"Sleep Apnea, Obstructive",disease,C08;C10,Disease or Syndrome,0.7,1.0,2016.0,2016.0,1,GWASCAT,NaN,NaN,NaN,NaN


In [18]:
disgenet_total.columns

Index(['snp_id', 'chromosome', 'position', 'dsi', 'dpi', 'diseaseid',
       'diseasename', 'diseasetype', 'diseaseclass', 'diseasesemantictype',
       'score', 'ei', 'yearinitial', 'yearfinal', 'nofpmids', 'source',
       'gene_id', 'gene_name', 'source_id', 'uniprot_acc'],
      dtype='object')

# Dividir la tabla total en distintos subsets: Genes; Diseases; Stats

In [20]:
disgenet_total_genes = disgenet_total[['snp_id', 'uniprot_acc', 'gene_name', 'gene_id', 'chromosome', 'position', 'source']]

In [22]:
disgenet_total_disease = disgenet_total[['snp_id', 'gene_name', 'gene_id', 'diseaseid', 'diseasename', 'diseasetype', 'diseaseclass', 'diseasesemantictype',
                                        'source']]

In [23]:
disgenet_total_genes

,snp_id,uniprot_acc,gene_name,gene_id,chromosome,position,source
0,rs1000005,NaN,NaN,NaN,21,33060745,GWASCAT
1,rs10000770,O15327,INPP4B,8821.0,4,142693109,GWASCAT
2,rs10000770,O15327,INPP4B,8821.0,4,142693109,GWASCAT
3,rs1000091588,P38935,IGHMBP2,3508.0,11,68935374,CLINVAR
4,rs1000091588,P38935,IGHMBP2,3508.0,11,68935374,CLINVAR
...,...,...,...,...,...,...,...
489665,rs999921351,Q9NQW8,CNGB3,54714.0,8,86628939,CLINVAR
489666,rs999943,Q14573,ITPR3,3710.0,6,33656956,GWASCAT;GWASDB
489667,rs999943,Q14573,ITPR3,3710.0,6,33656956,GWASCAT;GWASDB
489668,rs999944,NaN,NaN,NaN,2,64822719,GWASCAT
